SQL databases store data in tables with a strict predened schema and follow the
ACID properties.

Atomicity - A transaction is all-or-nothing (it either completes fully or not at

Consistency – Data always remains valid and follows dened rules.

Isolation – Transactions don’t interfere with each other.


Durability – Once data is saved, it won’t be lost, even if the system crashes.


In [1]:
import redis

In [2]:
# Redis connection
redis_cache = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

In [3]:
redis_cache

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=localhost,port=6379,db=0)>)>)>

DATABASE INDEX:

In [ ]:
CREATE TABLE employees(
    id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    department VARCHAR(50),
    salary DECIMAL(20)
    
);

In [ ]:
CREATE INDEX idx_last_name ON employees (last_name);

In [ ]:
SELECT * FROM employees WHERE last_name = 'Smith';

In [ ]:
CREATE INDEX idx_full_name ON employees (first_name, last_name);

In [ ]:
CREATE TABLE users (
    user_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    last_login TIMESTAMP,
    status VARCHAR(20) DEFAULT 'active'
);

without index sql looking for every row

In [ ]:
INSERT INTO users (name, email, last_login, status) VALUES
('Alice', 'alice@example.com', '2025-04-10 10:00:00', 'active'),
('Bob', 'bob@example.com', '2025-04-12 15:30:00', 'active'),
('Charlie', 'charlie@example.com', '2025-04-11 09:20:00', 'inactive'),
('David', 'david@example.com', '2025-04-13 12:45:00', 'active'),
('Eve', 'eve@example.com', '2025-04-09 08:15:00', 'active');

In [ ]:
SELECT * FROM users WHERE email = 'alice@example.com';

In [ ]:
SELECT * FROM users WHERE status = 'active' AND last_login >= '2025-04-12 00:00:00' ORDER BY last_login DESC;

with index

In [ ]:
-- Index on email for fast lookups
CREATE INDEX idx_users_email ON users (email);

-- Composite index on status and last_login for activity queries
CREATE INDEX idx_users_status_last_login ON users (status, last_login);

idx_users_email: B-tree index on email for exact match queries.
idx_users_status_last_login: Composite index supporting filtering by status and sorting by last_login.

In [ ]:
SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'users';

In [ ]:
CREATE DATABASE index_demo;

USE index_demo;

CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    age INT,
    INDEX idx_email (email)   -- Create index on email
);


REPLICATION

MySQL Master-Slave Replication (Manual)
✅ 1.1 Configure Master (MySQL on port 3306)

In [ ]:
CREATE USER 'replica'@'%' IDENTIFIED BY 'replica_pass';
GRANT REPLICATION SLAVE ON *.* TO 'replica'@'%';
FLUSH PRIVILEGES;

USE index_demo;
FLUSH TABLES WITH READ LOCK;
SHOW MASTER STATUS;


file and position

SAMANTA-140197S-bin.000365	15792	

Configure Replica (MySQL on port 3307)

In [ ]:
STOP SLAVE;

CHANGE MASTER TO
  MASTER_HOST='127.0.0.1',
  MASTER_PORT=3306,
  MASTER_USER='replica',
  MASTER_PASSWORD='replica_pass',
  MASTER_LOG_FILE='master-bin.000001',
  MASTER_LOG_POS=154;

START SLAVE;

SHOW SLAVE STATUS;


	127.0.0.1	replica	3306	60	SAMANTA-140197S-bin.000001	154	SAMANTA-140197S-relay-bin.000001	4	SAMANTA-140197S-bin.000001	No	Yes							0		0	154	157	None		0	No							No	13117	Fatal error: The replica I/O thread stops because source and replica have equal MySQL server ids; these ids must be different for replication to work (or the --replicate-same-server-id option must be used on replica but this does not always make sense; p...	0			1		mysql.slave_master_info	0		Replica has read all relay log; waiting for more updates	86400		250414 16:28:09						0					0	

delete replica

In [ ]:
STOP SLAVE;
RESET SLAVE ALL;
DROP USER 'replica'@'%';
FLUSH PRIVILEGES;


 Assign Unique server-id Values
🔧 Step 1: Edit MySQL Config Files
On Master (port 3306):

ini

In [ ]:
 Assign Unique server-id Values
🔧 Step 1: Edit MySQL Config Files
On Master (port 3306):

In [ ]:
[mysqld]
server-id=1
log-bin=mysql-bin


Replica (port 3307):

ini

In [ ]:
[mysqld]
server-id=2
relay-log=relay-log
read-only=1


vertical partitioning

In [ ]:
CREATE DATABASE IF NOT EXISTS userdb;
USE userdb;

-- Base table before partitioning
CREATE TABLE users (
    id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(50),
    email VARCHAR(100),
    password VARCHAR(255),
    bio TEXT,
    profile_picture TEXT,
    last_login DATETIME
);

-- Insert dummy data
INSERT INTO users (username, email, password, bio, profile_picture, last_login)
VALUES 
('alice', 'alice@example.com', 'pass123', 'Loves coding', '/images/alice.jpg', NOW()),
('bob', 'bob@example.com', 'secure456', 'DevOps fan', '/images/bob.jpg', NOW());


In [ ]:
CREATE TABLE user_basic (
    id INT PRIMARY KEY,
    username VARCHAR(50),
    email VARCHAR(100),
    password VARCHAR(255)
);

CREATE TABLE user_profile (
    user_id INT PRIMARY KEY,
    bio TEXT,
    profile_picture TEXT,
    last_login DATETIME,
    FOREIGN KEY (user_id) REFERENCES user_basic(id)
);

-- Migrate existing data
INSERT INTO user_basic (id, username, email, password) SELECT id, username, email, password FROM users;

INSERT INTO user_profile (user_id, bio, profile_picture, last_login) SELECT id, bio, profile_picture, last_login FROM users;

-- Optional: Drop original table
DROP TABLE users;
